In [1]:
import pandas as pd
import numpy as np
import requests
import json
import urllib.request
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

In [2]:
def title_scrap(url: str):
    url = url

    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        h3_tags = soup.find_all("h3", class_="blog_post_title my-2")

        titles = []

        for tag in h3_tags:
            if tag.text.strip():
                titles.append(tag.text.strip())

        df = pd.DataFrame({"Titles": titles})
    return df

In [3]:
df_title = title_scrap("https://www.6234.ir/")

In [4]:
def location_scrap(url: str):
    url = url

    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        h3_tags = soup.find_all("div", class_="blog_post_title my-2")

        titles = []

        for tag in h3_tags:
            if tag.text.strip():
                titles.append(tag.text.strip())

        df_location = pd.DataFrame({"Titles": titles})
    return df_location

In [5]:
df_location = location_scrap("https://www.6234.ir/")

In [6]:
def date_scrap(url: str):
    url = url

    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, "html.parser")

        h3_tags = soup.find_all("div", class_="theater-date my-2")

        titles = []

        for tag in h3_tags:
            if tag.text.strip():
                titles.append(tag.text.strip())

        df_date = pd.DataFrame({"Titles": titles})
    return df_date

In [7]:
df_date = date_scrap("https://www.6234.ir/")

In [8]:
def user_interaction_from_api(api: str):
    link = api
    urllib.request.urlretrieve(link, "log.xlsx")
    interaction = pd.read_excel("log.xlsx")

    return interaction

In [9]:
def user_iteraction_from_file(file_path: str):
    iteraction = pd.read_excel(file_path)
    return iteraction

In [10]:
def user_buy_history_from_api(api: str):
    link = api
    urllib.request.urlretrieve(link, "visitor.xlsx")
    buy_history = pd.read_excel("visitor.xlsx")

    return buy_history

In [11]:
def user_buy_history_from_file(file_path: str):

    buy_history = pd.read_excel(file_path)

    return buy_history

In [12]:
buy_history = pd.read_excel("./گزارش بلیط- نمونه.xlsx")

In [13]:
iteraction = pd.read_excel("./لاک بازدید- نمونه.xlsx")

In [14]:
def merge_df(
    title_scrap: pd.DataFrame,
    location_scrap: pd.DataFrame,
    date_scrap: pd.DataFrame,
    itraction,
    buy_history,
):
    merge_df = pd.DataFrame(
        {
            "Titles": title_scrap["Titles"],
            "Location": location_scrap["Titles"],
            "Date": date_scrap["Titles"],
        }
    )
    merge_df = pd.concat(
        [merge_df["Titles"], itraction["بازدید"], buy_history["رویداد"]]
    ).reset_index()
    merge_df.columns = ["index", "Titles"]
    return merge_df

In [15]:
merge_df = merge_df(df_title, df_location, df_date, iteraction, buy_history)

In [16]:
def iteraction_pre(df):
    df = df
    le = LabelEncoder()
    df["userId"] = le.fit_transform(df["نام و نام خانوادگی"])

    return df

In [17]:
df = iteraction_pre(iteraction)

In [18]:
np.unique(df["userId"])
M = df["userId"].nunique()
dict(zip(np.unique(df["userId"]), list(range(M))))

{0: 0, 1: 1, 2: 2}

In [19]:
def creat_X(df):


    M = df["userId"].nunique()

    N = df["بازدید"].nunique()


    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))

    item_mapper = dict(zip(np.unique(df["بازدید"]), list(range(N))))


    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))

    item_inv_mapper = dict(zip(list(range(N)), np.unique(df["بازدید"])))


    user_index = [user_mapper[i] for i in df["userId"]]

    item_indx = [item_mapper[i] for i in df["بازدید"]]


    X = csr_matrix((df["زمان تعامل(تانیه)"], (user_index, item_indx)), shape=(M, N))


    return X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper

In [20]:
X, user_mapper, item_mapper, user_inv_mapper, item_inv_mapper = creat_X(df)

In [21]:
def find_similar_item(iter_name, X, item_mapper, item_inv_mapper, k, metrics="cosine"):

    X = X.T
    neighbours_ids = []

    iter_ind = item_mapper[iter_name]
    iter_vec = X[iter_ind]
    if isinstance(iter_vec, (np.ndarray)):
        iter_vec = iter_vec.reshape(1, -1)
    # use k+1 since kNN output includes the user recommender of interest
    knn = NearestNeighbors(n_neighbors=k + 1, algorithm="brute", metric=metrics)
    knn.fit(X)
    neighbours = knn.kneighbors(iter_vec, return_distance=False)
    for i in range(0, k):
        n = neighbours.item(i)
        neighbours_ids.append(item_inv_mapper[n])
    neighbours_ids.pop(0)
    return neighbours_ids

In [22]:
def list_to_string(row):
    return " ".join(row)

In [23]:
df_ohe = merge_df["Titles"].str.split(" ").reset_index().astype("str")

In [24]:
df_ohe["Titles"] = df_ohe["Titles"].apply(list_to_string)

In [25]:
le = LabelEncoder()
merge_df["ohe"] = le.fit_transform(df_ohe["Titles"])

In [26]:
def vectorized_text(df: pd.DataFrame):
    vectorized = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
    X = vectorized.fit_transform(merge_df["Titles"])

    feature_names = vectorized.get_feature_names_out()
    one_hot_df = pd.DataFrame(X.toarray(), columns=feature_names)

    dfs = pd.concat([df, one_hot_df], axis=1)
    dfs.drop(columns=["Titles"], inplace=True)

    return dfs

In [27]:
dfs = vectorized_text(df_title)

In [28]:
def cosine_similioraty(
    dfs: pd.DataFrame, merge_pd, itraction, idx: str, n_recommendations: int = 1
):
    cosine_sim = cosine_similarity(dfs, dfs)
    iter_idx = dict(zip(merge_pd["Titles"], list(itraction.index)))
    idx = iter_idx[idx]
    n_recommendations = n_recommendations
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1 : (n_recommendations + 1)]
    similar_item = [i[0] for i in sim_scores]
    recomended = merge_pd["Titles"].iloc[similar_item]
    recomended = recomended.to_list()

    return recomended

In [37]:
cosine_sim = cosine_similarity(dfs, dfs)
iter_idx = dict(zip(merge_df["Titles"], list(iteraction.index)))
idx = iter_idx["کنسرت تست ( بدون انتخاب صندلی )"]
n_recommendations = 1
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1 : (n_recommendations + 1)]
similar_item = [i[0] for i in sim_scores]
recomended = merge_df["Titles"].iloc[similar_item]
recomended = recomended.to_list()

In [41]:
idx

0

In [29]:
user_dataframes = {}


for name, group_df in iteraction.groupby("نام و نام خانوادگی"):
    user_dataframes[name] = group_df.reset_index(drop=True)


for name, df in user_dataframes.items():
    globals()[f"{name}_dataframe"] = df

In [30]:
users = iteraction["نام و نام خانوادگی"].unique()

In [31]:
user_iter = {}
for i in users:
    user_it = (
        iteraction[iteraction["نام و نام خانوادگی"] == i][
            ["زمان تعامل(تانیه)", "بازدید"]
        ]
        .max()
        .reset_index()
        .T
    )
    user_it.columns = ["زمان تعامل(تانیه)", "بازدید"]
    user_it.drop(index="index", inplace=True)
    user_it["بازدید"]
    idx = user_it["بازدید"].to_list()[0]
    names = i
    iters = cosine_similioraty(dfs, merge_df, iteraction, idx=idx)
    user_dict = {names: iters}
    user_iter.update(user_dict)

In [32]:
user_iter

{'ناشناس': ['کنسرت نوشه ( ویژه بانوان )'],
 'سفردوست': ['کنسرت حامی'],
 'امیر نبی پور': ['دهمین همایش سالانه بانکداری الکترونیک']}

In [33]:
merge_df.shape

(307, 3)

In [34]:
iteraction.shape

(199, 9)

In [36]:
cosine_similarity(dfs, dfs).shape

(307, 307)